# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Settings

In [2]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# set up processed events folder if it doesn't already exist
if not os.path.exists('processed_events'):
    os.makedirs('processed_events')

#### Create a list of filepaths to process original event csv data files

In [3]:
for root, dirs, files in os.walk(filepath):
    dirs[:] = [d for d in dirs if d not in ['.ipynb_checkpoints']] 
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Then process the files to create a smaller csv file called event_data_new that will be used to load Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for raw_file in file_path_list:
    # reading csv file 
    with open(raw_file, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and appending it        
        for line in csvreader:
            full_data_rows_list.append(line) 

with open('processed_events/event_data_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating and connecting to a Cluster

In [5]:
# Connect to a Cassandra instance on local machine 
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create and set Keyspace

In [6]:
query = "CREATE KEYSPACE IF NOT EXISTS sparkifydb "
query = query + "WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}"

session.execute(query)
session.execute("USE sparkifydb")

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
For this query I have modelled a table called "song_in_session_details" as we are looking for the details of a song based on the session and it's listen position within that session.
The sessionId and itemInSession fields have been specified as the compound primary key for this table. The data will be partitioned by sessionId with itemInSession as the clustering column.
Using these two within the primary key allows the original question to be answered in an efficient way, and also provides a guarantee of uniqueness per row. 

I did not feel that inclusion of itemInSession within the partition key was necessary.

The rest of the fields included in this table are simply to allow the requested data to be accessed. Their inclusion within the primary key is not required.

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
I have modelled a table for this query called "user_session_song_details" as it provides the details of the song and user combos within a session. 
userId and sessionId were required within the primary key to allow the where clause on the target select query to efficiently access the data. This combination was specified as a composite partition key.

itemInSession was included as a clustering column to ensure rows were unique. This will also ensure that the table is sorted in ascending order by this value (within userId/sessionId)

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
I created a table called "song_listens" for this query as we are looking for users that have listened to a song by title.
The song title was used as the partition key as this is what is being used as a search term and seemed like the most sensible candidate out of the included columns.
I also added userId as a clustering column to ensure we can capture all the users that listened to a song, even if some have the same name. 

#### Create the tables (following dropping them if they already exist)

In [7]:
drop_query1 = "DROP TABLE IF EXISTS song_in_session_details;"
drop_query2 = "DROP TABLE IF EXISTS user_session_song_details;"
drop_query3 = "DROP TABLE IF EXISTS song_listens;"

create_query1 = "CREATE TABLE IF NOT EXISTS song_in_session_details "
create_query1 = create_query1 + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY(sessionId, itemInSession));"

create_query2 = "CREATE TABLE IF NOT EXISTS user_session_song_details "
create_query2 = create_query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession));"

create_query3 = "CREATE TABLE IF NOT EXISTS song_listens "
create_query3 = create_query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId));"

session.execute(drop_query1)
session.execute(drop_query2)
session.execute(drop_query3)
session.execute(create_query1)
session.execute(create_query2)
session.execute(create_query3)

#### Load data into all 3 tables in one pass to prevent repeated reads of the csv file

In [8]:
filepath = os.getcwd() + '/processed_events/event_data_new.csv'

with open(filepath, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_in_session_details (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, [int(line[8]), int(line[3]), line[0], line[9], float(line[5])])

        query1 = "INSERT INTO user_session_song_details (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query1, [int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]])

        query2 = "INSERT INTO song_listens (song, userId, firstName, lastName) "
        query2 = query2 + "VALUES (%s, %s, %s, %s)"
        session.execute(query2, [line[9], int(line[10]), line[1], line[4]])

### Answer original questions by querying the Cassandra tables

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
pd.DataFrame(list(session.execute("SELECT artist, song, length FROM song_in_session_details WHERE sessionId = 338 AND itemInSession = 4")))

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [10]:
pd.DataFrame(list(session.execute("SELECT artist, song, firstName, lastName FROM user_session_song_details WHERE userId = 10 AND sessionId = 182")))

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [11]:
pd.DataFrame(list(session.execute("SELECT firstName, lastName FROM song_listens WHERE song = 'All Hands Against His Own'")))

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [12]:
query = "DROP TABLE IF EXISTS song_in_session_details;"
query1 = "DROP TABLE IF EXISTS user_session_song_details;"
query2 = "DROP TABLE IF EXISTS song_listens;"

session.execute(query)
session.execute(query1)
session.execute(query2)

### Close the session and cluster connection¶

In [13]:
session.shutdown()
cluster.shutdown()